# Analyse du Data Drift — Credit Scoring

Ce notebook réalise l'analyse de la **dérive des données (data drift)** pour notre modèle de scoring crédit déployé en production.

**Approche :**

- **Référence** : distribution des données d'entraînement (stockée dans Postgres)
- **Production** : features envoyées à l'API pour chaque prédiction, horodatées et stockées dans Neon Postgres
- **Détection** : comparaison par fenêtre temporelle (KS test + Evidently AI)
- **Simulation** : le script `simulate_production_traffic.py` envoie des batches avec drift progressif

### Pourquoi pas train vs test ?

Comparer `train_preprocessed.csv` vs `test_preprocessed.csv` ne détecte pas du vrai drift car les deux proviennent du même dataset Kaggle. En production, le drift est **temporel** : la distribution des données évolue au fil du temps. Notre API horodate chaque prédiction — c'est cette dimension temporelle qu'on exploite.


---

## 1. Rappel : qu'est-ce que le Data Drift ?

Un modèle ML est entraîné sur des données historiques. En production, si les nouvelles données ont des **distributions différentes**, le modèle risque de perdre en fiabilité.

| Type              | Description                      | Exemple                                         |
| ----------------- | -------------------------------- | ----------------------------------------------- |
| **Drift graduel** | Distributions changent lentement | Inflation sur les montants de crédit            |
| **Drift soudain** | Changement brutal                | Nouveau segment de clientèle, crise             |
| **Feature shift** | Variables spécifiques changent   | Fournisseur de score externe modifie son calcul |

**Détection** : on compare la distribution de **référence** (entraînement) avec la distribution de **production** (prédictions récentes) via le test de Kolmogorov-Smirnov (KS). Si p-value < 0.05 → drift détecté.


---

## 2. Connexion à la base de données de production


In [ ]:
import json
import os
import sys
import warnings
from pathlib import Path

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from evidently import Report
from evidently.presets import DataDriftPreset

sys.path.insert(0, str(Path.cwd().parent))
from monitoring.drift import compute_drift_report, simulate_drift

warnings.filterwarnings("ignore", category=FutureWarning)

# Charger les variables d'environnement
load_dotenv(Path.cwd().parent / ".env")
DATABASE_URL = os.environ.get("DATABASE_URL", "")

ARTIFACTS_DIR = Path("../artifacts")
DATA_DIR = Path("../data")

print(f"DB connectée : {'Oui' if DATABASE_URL else 'Non'}")

In [ ]:
# Connexion à Neon Postgres
conn = psycopg2.connect(DATABASE_URL)

# Vérifier les données disponibles
stats = pd.read_sql(
    """
    SELECT
        count(*) AS total_predictions,
        min(created_at) AS first_prediction,
        max(created_at) AS last_prediction,
        avg(inference_time_ms) AS avg_latency_ms,
        avg(probability) AS avg_probability
    FROM predictions
""",
    conn,
)

print("=== Données de production en base ===")
print(f"Total prédictions : {stats['total_predictions'].iloc[0]}")
print(f"Première : {stats['first_prediction'].iloc[0]}")
print(f"Dernière : {stats['last_prediction'].iloc[0]}")
print(f"Latence moyenne : {stats['avg_latency_ms'].iloc[0]:.2f} ms")
print(f"Probabilité moyenne : {stats['avg_probability'].iloc[0]:.4f}")

---

## 3. Extraction des features de production

Chaque prédiction stockée en base contient les **419 features** envoyées à l'API (en JSONB), horodatées. On les extrait pour les comparer à la référence.


In [ ]:
# Charger les prédictions avec leurs features depuis Postgres
prod_raw = pd.read_sql(
    """
    SELECT features, probability, decision, inference_time_ms, created_at
    FROM predictions
    WHERE features IS NOT NULL
    ORDER BY created_at
""",
    conn,
)

print(f"Prédictions récupérées : {len(prod_raw)}")

# Extraire les features JSONB en colonnes
prod_features = pd.json_normalize(prod_raw["features"])
prod_features.index = prod_raw.index

print(f"Features extraites : {prod_features.shape[1]} colonnes")
prod_features.head(3)

In [ ]:
# Charger les données de référence (training data)
reference = pd.read_csv(DATA_DIR / "train_preprocessed.csv", nrows=5000)
drop_cols = [c for c in ["SK_ID_CURR", "TARGET"] if c in reference.columns]
reference = reference.drop(columns=drop_cols)

# Aligner les colonnes
common_cols = sorted(set(reference.columns) & set(prod_features.columns))
reference = reference[common_cols]
prod_aligned = prod_features[common_cols]

print(f"Référence : {reference.shape[0]} lignes, {reference.shape[1]} features")
print(f"Production : {prod_aligned.shape[0]} lignes, {prod_aligned.shape[1]} features")

---

## 4. Détection de drift : Référence vs Production

On compare la distribution des features d'entraînement (référence) avec celles reçues en production via le **test de Kolmogorov-Smirnov**.


In [ ]:
# KS test sur toutes les features (réutilise monitoring/drift.py)
drift_report = compute_drift_report(reference, prod_aligned, top_n=30)

n_analyzed = len(drift_report)
n_drifted = drift_report["drift_detected"].sum()
drift_pct = n_drifted / n_analyzed * 100

print(f"Features analysées : {n_analyzed}")
print(f"Features en drift : {n_drifted} ({drift_pct:.1f}%)")
print(
    f"\nStatus : {'ALERT' if drift_pct > 30 else 'WARNING' if drift_pct > 10 else 'OK'}"
)
print(f"\nTop 15 features par drift (KS statistic) :")
drift_report.head(15)

In [ ]:
# Visualiser les KS statistics
top_20 = drift_report.head(20)

fig, ax = plt.subplots(figsize=(12, 8))
colors = ["#D32F2F" if d else "#388E3C" for d in top_20["drift_detected"]]
bars = ax.barh(range(len(top_20)), top_20["ks_statistic"], color=colors)
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels([f[:30] for f in top_20["feature"]], fontsize=9)
ax.set_xlabel("KS Statistic")
ax.set_title("Top 20 Features — KS Test (Référence vs Production)", fontweight="bold")
ax.axvline(x=0.1, color="orange", linestyle="--", alpha=0.7, label="Seuil modéré")
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()

---

## 5. Rapport Evidently AI

Evidently AI génère un rapport interactif complet comparant les distributions de chaque feature.


In [ ]:
# Rapport Evidently : référence vs production
ev_report = Report([DataDriftPreset()])
snapshot = ev_report.run(reference, prod_aligned)

# Sauvegarder en HTML
report_path = Path("../monitoring/drift_report_evidently.html")
snapshot.save_html(str(report_path))
print(f"Rapport Evidently sauvegardé : {report_path}")

# Afficher dans le notebook
snapshot

---

## 6. Distributions comparées — Features critiques

Visualisons les distributions des features les plus importantes du modèle entre la référence et la production.


In [ ]:
# Top features du modèle par importance
model = joblib.load(ARTIFACTS_DIR / "model.pkl")
with open(ARTIFACTS_DIR / "feature_names.json") as f:
    feature_names = json.load(f)

importances = sorted(
    zip(feature_names, model.feature_importances_), key=lambda x: x[1], reverse=True
)
top_features = [name for name, _ in importances[:6] if name in common_cols]

print("Top features du modèle :")
for i, (name, imp) in enumerate(importances[:10], 1):
    print(f"  {i:2d}. {name:<35s} importance = {imp}")

In [ ]:
# Distributions comparées
n_plots = min(6, len(top_features))
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
axes = axes.flatten()

for i, feat in enumerate(top_features[:n_plots]):
    ax = axes[i]
    ax.hist(
        reference[feat].dropna(),
        bins=50,
        alpha=0.6,
        label="Référence (train)",
        color="#1D6A4B",
        density=True,
    )
    ax.hist(
        prod_aligned[feat].dropna(),
        bins=50,
        alpha=0.6,
        label="Production",
        color="#8B2D2D",
        density=True,
    )

    # KS stat pour cette feature
    ks_row = drift_report[drift_report["feature"] == feat]
    ks_val = ks_row["ks_statistic"].iloc[0] if len(ks_row) > 0 else "N/A"
    ax.set_title(f"{feat}\nKS = {ks_val}", fontsize=10, fontweight="bold")
    ax.legend(fontsize=8)

# Masquer les axes vides
for j in range(n_plots, len(axes)):
    axes[j].set_visible(False)

plt.suptitle(
    "Distributions comparées — Top Features (Référence vs Production)",
    fontsize=14,
    fontweight="bold",
    y=1.02,
)
plt.tight_layout()
plt.show()

---

## 7. Historique des rapports de drift

Les rapports de drift sont stockés dans la table `drift_reports` de Postgres, permettant de suivre l'évolution du drift au fil du temps.


In [ ]:
# Charger l'historique des rapports de drift
drift_history = pd.read_sql(
    """
    SELECT report_date, n_predictions, n_features_analyzed,
           n_features_drifted, drift_percentage, status
    FROM drift_reports
    ORDER BY report_date
""",
    conn,
)

print(f"Rapports de drift en base : {len(drift_history)}")
drift_history

In [ ]:
# Analyse opérationnelle : latence et distribution des scores
operational = pd.read_sql(
    """
    SELECT
        created_at,
        probability,
        decision,
        inference_time_ms
    FROM predictions
    ORDER BY created_at
""",
    conn,
)

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Latence au fil du temps
axes[0].plot(
    operational["created_at"],
    operational["inference_time_ms"],
    alpha=0.5,
    linewidth=0.5,
    color="#1B2A4A",
)
axes[0].axhline(
    y=operational["inference_time_ms"].quantile(0.95),
    color="red",
    linestyle="--",
    label="P95",
)
axes[0].set_title("Latence d'inférence (ms)", fontweight="bold")
axes[0].set_ylabel("ms")
axes[0].legend()

# Distribution des scores
approved = operational[operational["decision"] == "APPROVED"]["probability"]
refused = operational[operational["decision"] == "REFUSED"]["probability"]
axes[1].hist(approved, bins=30, alpha=0.7, label="APPROVED", color="#388E3C")
axes[1].hist(refused, bins=30, alpha=0.7, label="REFUSED", color="#D32F2F")
axes[1].axvline(x=0.494, color="black", linestyle="--", label="Seuil (0.494)")
axes[1].set_title("Distribution des scores", fontweight="bold")
axes[1].legend()

# Répartition des décisions
decision_counts = operational["decision"].value_counts()
axes[2].pie(
    decision_counts,
    labels=decision_counts.index,
    autopct="%1.1f%%",
    colors=["#388E3C", "#D32F2F"],
)
axes[2].set_title("Répartition des décisions", fontweight="bold")

plt.tight_layout()
plt.show()

print(f"\nMétriques opérationnelles :")
print(f"  Latence moyenne : {operational['inference_time_ms'].mean():.2f} ms")
print(f"  Latence P95 : {operational['inference_time_ms'].quantile(0.95):.2f} ms")
print(f"  Taux de refus : {(operational['decision'] == 'REFUSED').mean() * 100:.1f}%")

---

## 8. Simulation de drift et scripts de production

### Pipeline de monitoring en production

```
1. simulate_production_traffic.py  →  Envoie des batches à l'API avec drift progressif
   - Batch 1-3 : données propres (pas de drift)
   - Batch 4-6 : drift graduel (intensité 0.1, 0.2, 0.3)
   - Batch 7-8 : drift soudain (intensité 0.5, 0.7)

2. L'API stocke chaque prédiction + features dans Postgres (table predictions)

3. run_drift_detection.py  →  Compare les prédictions récentes vs référence
   - Fenêtre temporelle configurable (défaut: 24h)
   - KS test sur les top features
   - Résultat stocké dans drift_reports

4. Grafana / Streamlit  →  Visualisation temps réel
```

### Comment exécuter :

```bash
# 1. Lancer l'API
DATABASE_URL=... uvicorn api.app:app --port 8000

# 2. Simuler du trafic avec drift
python scripts/simulate_production_traffic.py --batch-size 50 --n-batches 8

# 3. Analyser le drift
DATABASE_URL=... python scripts/run_drift_detection.py
```


---

## 9. Points de vigilance et recommandations

### Résultats de l'analyse

| Aspect                 | Constat                                                                                 |
| ---------------------- | --------------------------------------------------------------------------------------- |
| **Stockage**           | Chaque prédiction stockée avec inputs (419 features JSONB), outputs, latence, timestamp |
| **Drift temporel**     | Le timestamp des prédictions permet une analyse par fenêtre temporelle                  |
| **Features critiques** | EXT*SOURCE*\*, AMT_CREDIT, DAYS_BIRTH doivent être monitorées en priorité               |
| **Simulation**         | Le drift simulé est correctement détecté, proportionnellement à l'intensité             |

### Seuils d'alerte

| Drift % | Status  | Action                       |
| ------- | ------- | ---------------------------- |
| < 10%   | OK      | Surveillance normale         |
| 10-30%  | WARNING | Investigation nécessaire     |
| > 30%   | ALERT   | Envisager un ré-entraînement |

### Architecture de monitoring

| Composant         | Rôle                                                                |
| ----------------- | ------------------------------------------------------------------- |
| **Neon Postgres** | Stockage : predictions, drift_reports, api_logs, training_reference |
| **Fluentd**       | Collecte des logs API structurés → Postgres                         |
| **Evidently AI**  | Rapports de drift détaillés (KS test, distributions)                |
| **Grafana**       | Dashboards : latence, volume, drift au fil du temps                 |
| **Streamlit**     | Dashboard interactif existant                                       |

### Limites

- **Drift ≠ dégradation** : un drift statistique ne signifie pas forcément que le modèle performe moins bien
- **Drift simulé vs réel** : nos simulations sont artificielles ; en production le drift est plus subtil
- **Pas de concept drift** : le test set Kaggle n'a pas de TARGET, donc on ne peut mesurer que le data drift, pas l'impact sur la performance
- **Conformité RGPD** : les features sont anonymisées, les rapports analysent des distributions statistiques (pas de données individuelles)


In [ ]:
conn.close()
print("Analyse de drift terminée.")
print("\nFichiers générés :")
print("  - monitoring/drift_report_evidently.html (rapport Evidently interactif)")
print("\nTables Postgres utilisées :")
print("  - predictions (inputs, outputs, latence, timestamp)")
print("  - drift_reports (rapports par fenêtre temporelle)")
print("  - training_reference (stats de référence)")
print("  - api_logs (logs structurés via Fluentd)")